In [1]:
# --- Imports ---
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import preprocess as pre

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import xgboost as xgb

In [2]:
# --- CSV ---
ais_train = pd.read_csv("../ais_train.csv", sep="|")
ais_test = pd.read_csv("../ais_test.csv", sep=",")

vessels = pd.read_csv("../vessels.csv", sep="|")
ports = pd.read_csv("../ports.csv", sep="|")
schedules = pd.read_csv("../schedules_to_may_2024.csv", sep="|")

In [9]:
# --- Data Preprocessing --- 
uniqueVesselId = ais_train["vesselId"].unique()
ais_train_processed_list = []

for id in uniqueVesselId:
    #Isolate a single boat and clean
    boat = ais_train[ais_train["vesselId"] == id]
    boat = pre.singleBoatCleanup(boat, False)

    #Appending rows to list
    for _, row in boat.iterrows():
        ais_train_processed_list.append(row.to_dict())

ais_train_processed = pd.DataFrame(ais_train_processed_list).sort_values("time").reset_index(drop=True)

ais_train_processed.head()

ais_train_processed.to_csv("ais_train_processed.csv", sep="|", index=False)



c:\Users\marti\Desktop\ML Prosjekt\Untitled Folder\preprocess.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boat["time"] = boat["time"].astype("datetime64[ns]")
c:\Users\marti\Desktop\ML Prosjekt\Untitled Folder\preprocess.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boat["drift"] = boat["cog"] - boat["heading"]
c:\Users\marti\Desktop\ML Prosjekt\Untitled Folder\preprocess.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind